# Activity 4.1 - Cleaning Walmart Data the OpenRefine Way

In this activity, you will practice what you learned in Lecture 4.5 by cleaning up a data set containing information on various Walmart locations.

In [11]:
import pandas as pd
from dfply import *

#### Initial Tasks

1. Try to read in the `./data/Walmart_United_States_&_Canada.csv` file and verify that you get an encoding error.  This means that the [character encoding](https://en.wikipedia.org/wiki/Character_encoding) isn't the default of `utf-8`.  The easiest way to fix this is to open and save the file in Visual Studio Code.

In [12]:
walmart = pd.read_csv('./data/Walmart_United_States_&_Canada.csv', names=['Lat', 'Long', 'Store', 'Address'])
walmart.head()

,Lat,Long,Store,Address
0,-114.005671,51.262567,"Walmart Supercentre; #1050,","2881 Main St SW,Airdrie ,AB T4B 3G5,(403) 945-..."
1,-111.900542,50.577939,"Walmart Supercentre; #3658,","917 3rd St W,Brooks ,AB T1R 1L5,(403) 793-2111"
2,-114.039133,51.107253,"Walmart Supercentre; #3013,","1110 57th Ave NE,Calgary ,(NOP),AB T2E 9B7,(40..."
3,-114.138488,51.040871,"Walmart Supercentre; #3009,Gas,","1212 37 St SW,Calgary ,(NOP),AB T3C 1S3,(403) ..."
4,-114.028603,50.930551,"Walmart; #1144,","1221 Canyon Meadows Dr SE,Calgary ,AB T2J 6G2,..."


2. Read in the data to verify that the encoding is fixed, but that there are two more problems.  What are they?

<font color="yellow"> Columns were combined. The rows were reading as headers. </font>

3. Take another look at the file in VS Code and determine solutions to the two/three issues, then read in the data correctly by passing `pd.read_csv` the correct defaults for this data. **Note.** Leave the `"` in place for now, as they serve an important role here!

In [15]:
walmart['Address'].str.split(',', expand=True)

,0,1,2,3,4,5
0,2881 Main St SW,Airdrie,AB T4B 3G5,(403) 945-1295,None,None
1,917 3rd St W,Brooks,AB T1R 1L5,(403) 793-2111,None,None
2,1110 57th Ave NE,Calgary,(NOP),AB T2E 9B7,(403) 730-0990,None
3,1212 37 St SW,Calgary,(NOP),AB T3C 1S3,(403) 242-2205,None
4,1221 Canyon Meadows Dr SE,Calgary,AB T2J 6G2,(403) 225-6638,None,None
...,...,...,...,...,...,...
6811,2390 E Cedar St; I-80 Exit 214,Rawlins,WY,82301,,(307) 417-3001
6812,1733 N Federal Blvd,Riverton,WY,82501,,(307) 856-3261
6813,201 Gateway Blvd; I-80 Exit 102,Rock Springs,WY,82901,,(307) 362-1957
6814,1695 Coffeen Ave; I-90 Exit 25,Sheridan,WY,82801,(NOP),(307) 674-6492


In [9]:
walmart['Store'].str.split(';', expand=True)


,0,1
0,Walmart Supercentre,"#1050,"
1,Walmart Supercentre,"#3658,"
2,Walmart Supercentre,"#3013,"
3,Walmart Supercentre,"#3009,Gas,"
4,Walmart,"#1144,"
...,...,...
6811,Walmart Supercenter,"#4471,Gas,"
6812,Walmart Supercenter,"#1457,"
6813,Walmart Supercenter,"#1461,"
6814,Walmart Supercenter,"#1508,"


<font color="blue"> Your thoughts here </font>

In [19]:
#Your code here

## Cleaning up the store information.

As hinted at above, the presence of the `"` meant the two of the columns--one containing the store type/number and the other contain the address/phone number--are combined together.  This was done because some of these entries have a different number of variables.  For example, the store type/number column sometimes occasionally `Gas`.

In this part of the activity, you should apply the iterative OpenRefine approach to separate the information in the store column.

**Warning!** There is one entry that doesn't follow the same pattern as the rest.  You won't find this entry unless you carefully define/fix/eliminate patterns.

In [21]:
!pip install more_itertools

In [22]:
from more_dfply import case_when, ifelse
from more_dfply.facets import text_facet, text_filter

# Your code here.

In [24]:
# View cell

(walmart
>> select(X.Store)
>> filter_by(text_filter(X.Store, 'Gas/Diesel'))
>> filter_by(text_filter(X.Store, 'Gas'))
)

,Store
70,"Murphy: USA; #7687,Gas/Diesel,"
71,"Walmart Supercenter; #0423,Gas/Diesel,"
75,"Walmart Supercenter; #1091,Gas/Diesel,"
76,"Murphy: USA; #7394,Gas/Diesel,"
79,"Walmart Supercenter; #0306,Gas/Diesel,"
...,...
6750,"Sam's Club; #6535,Gas/Diesel,"
6772,"Walmart Supercenter; #2474,Gas/Diesel,"
6782,"Sam's Club; #4936,Gas/Diesel,"
6790,"Sam's Club; #6457,Gas/Diesel,"


In [26]:
# Transform cell

(walmart
>> select(X.Store)
>> mutate(fuel = case_when((text_filter(X.Store, 'Gas/Diesel'),
                                                "Gas/Diesel"),
                           (text_filter(X.Store, 'Gas'),
                           "Gas"),
                           (True, "None") 
),

Store = X.Store.replace('(Gas/Diesel)?,', '', regex=True)

)
)

,Store,fuel
0,Walmart Supercentre; #1050,None
1,Walmart Supercentre; #3658,None
2,Walmart Supercentre; #3013,None
3,Walmart Supercentre; #3009Gas,Gas
4,Walmart; #1144,None
...,...,...
6811,Walmart Supercenter; #4471Gas,Gas
6812,Walmart Supercenter; #1457,None
6813,Walmart Supercenter; #1461,None
6814,Walmart Supercenter; #1508,None


In [37]:
(walmart
>> select(X.Store)
>> filter_by(~text_filter(X.Store, 'Gas/Diesel'))
>> filter_by(~text_filter(X.Store, 'Gas'))
>> mutate(Store = X.Store.replace('Gas(/Diesel)?,' '', regex=True))
>> filter_by(~text_filter(X.Store, 'Walmart( Supercentre| Supercentre)?; #\d{4},', regex=True))
>> mutate(Store = X.Store.replace(r'Wm (.*)', r'Walmart \1', regex=True)))


/home/samwooden/.pyenv/versions/anaconda3-2022.05/lib/python3.9/site-packages/more_dfply/facets.py:30: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  return col.str.contains(pattern, case=case, regex=regex, na=na)


,Store
61,"Walmart Supercenter; #2070,"
62,"Walmart Supercenter; #4359,"
63,"Walmart Supercenter; #2071,"
64,"Walmart Supercenter; #2188,"
65,"Walmart Supercenter; #2722,"
...,...
6809,"Walmart Supercenter; #1485,"
6810,"Walmart Supercenter; #1412,"
6812,"Walmart Supercenter; #1457,"
6813,"Walmart Supercenter; #1461,"


In [35]:
walmart['Store'].str.cat(walmart[['Store Type', 'Store Number']], sep=';')

0       Walmart Supercentre; #1050,;Walmart Supercentr...
1       Walmart Supercentre; #3658,;Walmart Supercentr...
2       Walmart Supercentre; #3013,;Walmart Supercentr...
3       Walmart Supercentre; #3009,Gas,;Walmart Superc...
4                         Walmart; #1144,;Walmart; #1144,
                              ...                        
6811    Walmart Supercenter; #4471,Gas,;Walmart Superc...
6812    Walmart Supercenter; #1457,;Walmart Supercente...
6813    Walmart Supercenter; #1461,;Walmart Supercente...
6814    Walmart Supercenter; #1508,;Walmart Supercente...
6815    Walmart Supercentre; #3191,;Walmart Supercentr...
Name: Store, Length: 6816, dtype: object

In [31]:
walmart[['Store Type', 'Store Number']] = walmart['Store'].str.split(';', expand=True)
walmart.head()

,Lat,Long,Store,Address,Store Type,Store Number
0,-114.005671,51.262567,"Walmart Supercentre; #1050,","2881 Main St SW,Airdrie ,AB T4B 3G5,(403) 945-...",Walmart Supercentre,"#1050,"
1,-111.900542,50.577939,"Walmart Supercentre; #3658,","917 3rd St W,Brooks ,AB T1R 1L5,(403) 793-2111",Walmart Supercentre,"#3658,"
2,-114.039133,51.107253,"Walmart Supercentre; #3013,","1110 57th Ave NE,Calgary ,(NOP),AB T2E 9B7,(40...",Walmart Supercentre,"#3013,"
3,-114.138488,51.040871,"Walmart Supercentre; #3009,Gas,","1212 37 St SW,Calgary ,(NOP),AB T3C 1S3,(403) ...",Walmart Supercentre,"#3009,Gas,"
4,-114.028603,50.930551,"Walmart; #1144,","1221 Canyon Meadows Dr SE,Calgary ,AB T2J 6G2,...",Walmart,"#1144,"


## Preview of Coming Attractions

In this module's homework assignment, you will continue to clean up this data set.